# Constraint Programming Task Scheduler

If you have a bunch of tasks to do in a week, and you have only so many hours in each day of the week, how do you figure out what to do and when?

This is a IPython notebook using constraint programming to answer that question.

- Tasks are defined with work effort in hours
- The schedule for the week is defined by assigning each day with available free hours. 


In [34]:
from constraint import *
from itertools import *

# Configuration and Setup

In [35]:
# Configure day naming constants
M,T,W,TH,F,SAT,SUN = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

In [36]:
# Configure number of free hours each day
#
# Sample :
#   free_time[W] = 3  # Sets wednesday to 3 free hours

free_time = {}
free_time[M]  = 5  # Monday
free_time[T]  = 4  # Tuesday
free_time[W]  = 2  # Wednesday
free_time[TH] = 5  # Thursday
free_time[F]  = 1  # Friday
free_time[SAT]  = 0  # Saturday
free_time[SUN]  = 2  # Sunday

In [37]:
# Configure tasks to complete this week
#
# Sample tasks
#   tasks['name'] = { 'effort': 1 }

tasks = {}
tasks['laundry'] = { 'effort': 2 , 'due': M }
tasks['clean dishes'] = { 'effort': 1, 'due': T }
tasks['pay bills'] = { 'effort': 2 , 'due': W }
tasks['mow the lawn'] = { 'effort': 3 , 'due' : TH}
tasks['write blog post'] = { 'effort': 1 , 'due': SUN }
tasks['wash the car'] = { 'effort': 3 }

# Precalculate intermediate values

In [38]:
# Calculate helper variables
# Descriptions below
total_free_hours = 0
hour_to_day = {}
order_of_days = [M,T,W,TH,F,SAT,SUN]

for day in order_of_days:
    for hour in range(0,free_time[day]):
        hour_to_day[total_free_hours] = day
        total_free_hours += 1

hours_in_week = range(0,total_free_hours)

In [39]:
# hour_to_day is a mapping of hour in week to a day
hour_to_day

{0: 'monday',
 1: 'monday',
 2: 'monday',
 3: 'monday',
 4: 'monday',
 5: 'tuesday',
 6: 'tuesday',
 7: 'tuesday',
 8: 'tuesday',
 9: 'wednesday',
 10: 'wednesday',
 11: 'thursday',
 12: 'thursday',
 13: 'thursday',
 14: 'thursday',
 15: 'thursday',
 16: 'friday',
 17: 'sunday',
 18: 'sunday'}

In [40]:
# total_free_hours is a sum of all the free hours this week 
total_free_hours

19

In [41]:
# hours_in_week is an incremental array of free hours this week
hours_in_week

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [42]:
# list of all tasks
all_tasks = tasks.keys()
all_tasks

['write blog post',
 'laundry',
 'wash the car',
 'clean dishes',
 'pay bills',
 'mow the lawn']

# Defining the constraint solver

In [43]:
problem = Problem()
# task.keys() , i.e. ['laundry', 'pay bills', ...] are the variables
# hours_in_week, i.e. [0,1,2,3,...] are the possible values for the variables
# 
# The constraint solver will try assigning different hours of the week
# to the variables, and reject all those that do not meet the constraints
problem.addVariables(all_tasks,hours_in_week)

## Add Constraint: Two tasks cannot begin at the same time

In [44]:
# Add a constraint so that no task begins on an identical hour to another task
problem.addConstraint(AllDifferentConstraint())

## Add Constraint: A task needs enough time to complete before the week ends

In [45]:
# Iterate over each task
# Add a constraint that validates the start time of a task allows enough time to complete it
for task_name in all_tasks:
    problem.addConstraint(lambda start: start <= total_free_hours - tasks[task_name]['effort'], [task_name])

## Add Constraint: Two tasks can't be worked on at the same time

In [46]:
# Iterate over each set of possible tasks
# Add a constraint that validates the start time + work effort is enough to complete the task
# before the next task should begin
for two_tasks in permutations(all_tasks,2):
    problem.addConstraint(lambda first_task, second_task: first_task + tasks[two_tasks[0]]['effort'] <= second_task if first_task < second_task else True, [two_tasks[0], two_tasks[1]])
    problem.addConstraint(lambda first_task, second_task: second_task + tasks[two_tasks[1]]['effort'] <= first_task if first_task > second_task else True, [two_tasks[0], two_tasks[1]])

## Add Constraint: A task can't be worked past a certain point, or else it would not meet it's due date

In [47]:
# Return the index of the week given a day
#
# Sample: getDayIndex(M) => 0
#         getDayIndex(T) => 1
def getDayIndex(day):
    return order_of_days.index(day,0,len(order_of_days))

In [48]:
# getMaxStartHour
# Get the maximum hour in the week the task can be assigned to.
# If the task were started in an hour past this number, it would violate either
# effort to complete or the due date
def getMaxStartHour(task_name):
    if tasks[task_name].has_key('due'):
        max_hour = 0
        for hour in hours_in_week:
            #print hour, task_name, getDayIndex(hour_to_day[hour + tasks[task_name]['effort']-1]), getDayIndex(tasks[task_name]['due']) + 1
            if getDayIndex(hour_to_day[hour + tasks[task_name]['effort']-1]) < getDayIndex(tasks[task_name]['due']) + 1:
                continue
            else:
                return hour
        return max(hours_in_week)
    else:
        return max(hours_in_week) - tasks[task_name]['effort'] + 1

# Print max start hours for each task
print "Max start hours for each task"
print "============================="
for task in tasks:
    print task,getMaxStartHour(task)

Max start hours for each task
write blog post 18
laundry 4
wash the car 16
clean dishes 9
pay bills 10
mow the lawn 14


In [49]:
# Add Constraint: Task must be finished by due date
for t in all_tasks:
    if tasks[t].has_key('due'):
        max_start_hour = getMaxStartHour(t)
        problem.addConstraint(InSetConstraint(range(0,max_start_hour)),[t])

# Solution

## Possible Number of Solutions

In [50]:
len(problem.getSolutions())

144

## Get Solution

In [51]:
# Get a solution
solution = problem.getSolution()
solution

{'clean dishes': 0,
 'laundry': 3,
 'mow the lawn': 10,
 'pay bills': 7,
 'wash the car': 13,
 'write blog post': 16}

## Pretty Print Solution

In [52]:
def print_schedule():
    prev_day = None
    for hour in hour_to_day.keys():
        # Print day
        if hour_to_day[hour] != prev_day :
            print "\n","== "+ hour_to_day[hour] + " =="
            prev_day = hour_to_day[hour]
        # Print task name
        current_task = None
        for task_name in solution:
            if (hour == solution[task_name]):
                if ( (current_task == None) or (solution[current_task] < solution[task_name]) ):
                    current_task = task_name
        if(current_task != None):
            print "  + ",current_task, tasks[current_task]
        else:
            print "  -"

# Print Schedule

The following is a possible schedule that will meet the constraints placed on the tasks.

Tasks are assumed to be started and flow through to the next date.
That is why you may see a task start on a certain day, but get completed on another.

In [53]:
print_schedule()


== monday ==
  +  clean dishes {'effort': 1, 'due': 'tuesday'}
  -
  -
  +  laundry {'effort': 2, 'due': 'monday'}
  -

== tuesday ==
  -
  -
  +  pay bills {'effort': 2, 'due': 'wednesday'}
  -

== wednesday ==
  -
  +  mow the lawn {'effort': 3, 'due': 'thursday'}

== thursday ==
  -
  -
  +  wash the car {'effort': 3}
  -
  -

== friday ==
  +  write blog post {'effort': 1, 'due': 'sunday'}

== sunday ==
  -
  -
